In [40]:
import pandas as pd
import numpy as np

In [41]:
result_cot_general = pd.read_csv('experiment_results/gpt-turbo_multiarith_cot_general_8shot.csv', index_col=0)

result_arith_general_new = pd.read_csv('experiment_results/gpt-turbo_multiarith_arithcot_general_8shot_new.csv', index_col=0)

result_arith_general_old = pd.read_csv('experiment_results/gpt-turbo_multiarith_arithcot_general_8shot_old.csv', index_col=0)


In [42]:
def acc(results):
    return np.mean(results['answer'] == results['response_answer'])

def missing(results):
    np.sum(results['response_answer'].isnull())

In [43]:
print(acc(result_cot_general))
print(acc(result_arith_general_old))
print(acc(result_arith_general_new))

0.975
0.9483333333333334
0.9483333333333334


## Two different prompts

In [38]:
result_arith_general_old.columns = [i + "_old" for i in result_arith_general_old.columns]

In [39]:
c_arith_general = pd.concat([result_arith_general, result_arith_general_old], axis = 1)

In [42]:
c_arith_general_errors = c_arith_general[['answer','response_answer','response_answer_old']]

In [46]:
c_arith_general[c_arith_general.response_answer != c_arith_general.response_answer_old].count()

question               87
answer                 87
response               87
mapping                87
response_answer        48
equations              87
question_old           87
answer_old             87
response_old           87
mapping_old            87
response_answer_old    54
equations_old          87
dtype: int64

In [51]:
result_arith_general_old[result_arith_general_old['response_answer_old'].isnull()].to_csv('temp.csv')

## cot vs. arithcot

In [44]:
result_arith_general_old.columns = [i + '_old' for i in result_arith_general_old.columns]

In [45]:
result_arith_general_new.columns = [i + '_new' for i in result_arith_general_new.columns]

In [46]:
cot_arithcot = pd.concat([result_cot_general, result_arith_general_old, result_arith_general_new], axis = 1)

In [47]:
cot_arithcot.drop(columns = ['question_old', 'answer_old', 'question_new', 'answer_new'], inplace = True)

In [49]:
cot_arithcot['median_answer'] = -1

In [65]:
for i in range(0,600):
    a1 = cot_arithcot['response_answer'][i]
    a2 = cot_arithcot['response_answer_new'][i]
    a3 = cot_arithcot['response_answer_old'][i]

    median_answer = np.nanmedian([a1,a2,a3])
    cot_arithcot['median_answer'][i] = median_answer

<ipython-input-65-b9da98262b74>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cot_arithcot['median_answer'][i] = median_answer


In [66]:
np.mean(cot_arithcot['answer'] == cot_arithcot['median_answer'])

0.9816666666666667

In [67]:
changes = pd.DataFrame(cot_arithcot)

for i in range(0,600):
    if(changes['answer'][i] != changes['response_answer'][i] or
       changes['answer'][i] != changes['response_answer_new'][i] or
       changes['answer'][i] != changes['response_answer_old'][i]):
        pass
    else:
        changes.drop(index = i, inplace=True)


In [68]:
changes = changes[["question", "answer", 'median_answer',"response_answer", "response_answer_old","response_answer_new", "mapping_old", "response","response_answer_old","response_old","response_new"]]

In [69]:
changes.to_csv("temp.csv")
